# Mini-ChatGPT in Google Colab

This notebook implements a mini version of ChatGPT using The Pile dataset from Hugging Face.

## Features:
- Streams a subset of The Pile dataset
- Uses GPT-2 tokenizer for text processing
- Implements a small GPT model (2-4 transformer layers)
- Trains with AdamW optimizer
- Provides a simple chat interface

## Requirements:
- Google Colab or local Jupyter environment
- GPU recommended for training

**Note**: This is an educational implementation optimized for Colab's resource constraints.

## 1. Install Required Packages

In [ ]:
# Install required packages
!pip install transformers datasets torch accelerate tqdm numpy

# Import necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import GPT2Tokenizer, AdamW, get_linear_schedule_with_warmup
from datasets import load_dataset
import numpy as np
from tqdm import tqdm
import math
import random
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

## 2. Load and Stream The Pile Dataset

In [ ]:
# Load GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # Set padding token

print(f"Tokenizer vocabulary size: {len(tokenizer)}")
print(f"EOS token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")
print(f"PAD token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")

In [ ]:
# Load a subset of The Pile dataset
# We'll use a smaller subset for Colab compatibility
print("Loading The Pile dataset...")
try:
    # Load the train split with streaming=True for memory efficiency
    dataset = load_dataset("EleutherAI/pile", streaming=True, split="train")
    print("Successfully loaded The Pile dataset")
except Exception as e:
    print(f"Error loading The Pile: {e}")
    print("Falling back to a smaller text dataset...")
    # Fallback to a smaller dataset if The Pile is not accessible
    dataset = load_dataset("wikitext", "wikitext-2-raw-v1", streaming=True, split="train")
    print("Using WikiText-2 as fallback dataset")

# Take a small sample for demonstration (adjust as needed for your Colab session)
sample_size = 5000  # Number of texts to use for training
print(f"Using {sample_size} samples for training")

## 3. Data Processing and Tokenization

In [ ]:
class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=256):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.texts = []
        
        print("Tokenizing texts...")
        for text in tqdm(texts, desc="Processing texts"):
            if isinstance(text, dict):
                # Handle different dataset formats
                text_content = text.get('text', '') or text.get('content', '')
            else:
                text_content = str(text)
            
            # Skip very short texts
            if len(text_content.strip()) < 50:
                continue
                
            # Tokenize and truncate
            tokens = tokenizer.encode(text_content, max_length=max_length, truncation=True)
            if len(tokens) > 10:  # Only keep texts with reasonable length
                self.texts.append(tokens)
        
        print(f"Processed {len(self.texts)} valid texts")
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        tokens = self.texts[idx]
        
        # For language modeling, input and target are the same, shifted by one
        if len(tokens) > 1:
            input_ids = tokens[:-1]
            target_ids = tokens[1:]
        else:
            input_ids = tokens
            target_ids = tokens
        
        # Pad to max_length
        input_ids = input_ids + [tokenizer.pad_token_id] * (self.max_length - 1 - len(input_ids))
        target_ids = target_ids + [tokenizer.pad_token_id] * (self.max_length - 1 - len(target_ids))
        
        return {
            'input_ids': torch.tensor(input_ids[:self.max_length-1], dtype=torch.long),
            'target_ids': torch.tensor(target_ids[:self.max_length-1], dtype=torch.long)
        }

# Collect sample texts
print("Collecting sample texts from dataset...")
texts = []
for i, example in enumerate(dataset):
    if i >= sample_size:
        break
    texts.append(example)
    if (i + 1) % 1000 == 0:
        print(f"Collected {i + 1} texts")

print(f"Total texts collected: {len(texts)}")

In [ ]:
# Create dataset and dataloader
max_seq_length = 128  # Smaller sequence length for Colab
batch_size = 8  # Small batch size for memory efficiency

train_dataset = TextDataset(texts, tokenizer, max_length=max_seq_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

print(f"Dataset size: {len(train_dataset)}")
print(f"Number of batches: {len(train_dataloader)}")

# Show a sample batch
sample_batch = next(iter(train_dataloader))
print(f"\nSample batch shape:")
print(f"Input IDs: {sample_batch['input_ids'].shape}")
print(f"Target IDs: {sample_batch['target_ids'].shape}")

# Decode a sample to verify
sample_text = tokenizer.decode(sample_batch['input_ids'][0], skip_special_tokens=True)
print(f"\nSample text preview: {sample_text[:200]}...")

## 4. Mini GPT Model Definition

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        assert d_model % num_heads == 0
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        
        attention_weights = F.softmax(scores, dim=-1)
        output = torch.matmul(attention_weights, V)
        return output
    
    def forward(self, x, mask=None):
        batch_size, seq_len, d_model = x.size()
        
        Q = self.W_q(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_k(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_v(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        
        attention_output = self.scaled_dot_product_attention(Q, K, V, mask)
        attention_output = attention_output.transpose(1, 2).contiguous().view(
            batch_size, seq_len, d_model
        )
        
        output = self.W_o(attention_output)
        return output

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        return self.linear2(self.relu(self.linear1(x)))

class TransformerBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.attention = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = FeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        # Self-attention with residual connection
        attention_output = self.attention(x, mask)
        x = self.norm1(x + self.dropout(attention_output))
        
        # Feed-forward with residual connection
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        
        return x

class MiniGPT(nn.Module):
    def __init__(self, vocab_size, d_model=256, num_heads=8, num_layers=4, d_ff=1024, max_seq_len=512, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.max_seq_len = max_seq_len
        
        # Embeddings
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        self.position_embedding = nn.Embedding(max_seq_len, d_model)
        
        # Transformer blocks
        self.transformer_blocks = nn.ModuleList([
            TransformerBlock(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])
        
        # Output layer
        self.ln_f = nn.LayerNorm(d_model)
        self.lm_head = nn.Linear(d_model, vocab_size, bias=False)
        
        self.dropout = nn.Dropout(dropout)
        
        # Initialize weights
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def create_causal_mask(self, seq_len):
        mask = torch.tril(torch.ones(seq_len, seq_len))
        return mask.view(1, 1, seq_len, seq_len)
    
    def forward(self, input_ids):
        batch_size, seq_len = input_ids.size()
        
        # Create position indices
        position_ids = torch.arange(seq_len, dtype=torch.long, device=input_ids.device)
        position_ids = position_ids.unsqueeze(0).expand(batch_size, seq_len)
        
        # Embeddings
        token_embeds = self.token_embedding(input_ids)
        position_embeds = self.position_embedding(position_ids)
        x = self.dropout(token_embeds + position_embeds)
        
        # Create causal mask
        mask = self.create_causal_mask(seq_len).to(input_ids.device)
        
        # Pass through transformer blocks
        for transformer_block in self.transformer_blocks:
            x = transformer_block(x, mask)
        
        # Final layer norm and output projection
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        return logits

# Model configuration
model_config = {
    'vocab_size': len(tokenizer),
    'd_model': 256,  # Hidden size
    'num_heads': 8,
    'num_layers': 4,  # Number of transformer layers
    'd_ff': 1024,  # Feed-forward dimension
    'max_seq_len': max_seq_length,
    'dropout': 0.1
}

# Initialize model
model = MiniGPT(**model_config)
model = model.to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Model configuration: {model_config}")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Model size: ~{total_params * 4 / 1024 / 1024:.1f} MB")

## 5. Training Configuration and Loop

In [ ]:
# Training configuration
num_epochs = 2  # Limited for Colab
learning_rate = 5e-4
weight_decay = 0.01
warmup_steps = 100
max_grad_norm = 1.0

# Initialize optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=warmup_steps, 
    num_training_steps=total_steps
)

print(f"Training configuration:")
print(f"Epochs: {num_epochs}")
print(f"Learning rate: {learning_rate}")
print(f"Batch size: {batch_size}")
print(f"Total training steps: {total_steps}")
print(f"Warmup steps: {warmup_steps}")

# Loss function
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

def calculate_perplexity(loss):
    return math.exp(loss)

print("\nStarting training...")
print("="*50)

In [ ]:
# Training loop
model.train()
total_loss = 0
step = 0

for epoch in range(num_epochs):
    epoch_loss = 0
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
    
    for batch in progress_bar:
        # Move batch to device
        input_ids = batch['input_ids'].to(device)
        target_ids = batch['target_ids'].to(device)
        
        # Forward pass
        optimizer.zero_grad()
        logits = model(input_ids)
        
        # Calculate loss
        # Reshape for cross entropy: (batch_size * seq_len, vocab_size)
        logits_flat = logits.view(-1, logits.size(-1))
        targets_flat = target_ids.view(-1)
        
        loss = criterion(logits_flat, targets_flat)
        
        # Backward pass
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        
        # Update weights
        optimizer.step()
        scheduler.step()
        
        # Update metrics
        epoch_loss += loss.item()
        total_loss += loss.item()
        step += 1
        
        # Update progress bar
        current_lr = scheduler.get_last_lr()[0]
        perplexity = calculate_perplexity(loss.item())
        progress_bar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'ppl': f'{perplexity:.2f}',
            'lr': f'{current_lr:.2e}'
        })
        
        # Log every 100 steps
        if step % 100 == 0:
            avg_loss = total_loss / step
            print(f"\nStep {step}: avg_loss = {avg_loss:.4f}, perplexity = {calculate_perplexity(avg_loss):.2f}")
    
    # End of epoch summary
    avg_epoch_loss = epoch_loss / len(train_dataloader)
    epoch_perplexity = calculate_perplexity(avg_epoch_loss)
    
    print(f"\nEpoch {epoch+1} completed:")
    print(f"Average loss: {avg_epoch_loss:.4f}")
    print(f"Perplexity: {epoch_perplexity:.2f}")
    print("-" * 50)

print("\nTraining completed!")
final_avg_loss = total_loss / step
final_perplexity = calculate_perplexity(final_avg_loss)
print(f"Final average loss: {final_avg_loss:.4f}")
print(f"Final perplexity: {final_perplexity:.2f}")

## 6. Chat Interface Implementation

In [ ]:
def generate_text(model, tokenizer, prompt, max_length=100, temperature=0.8, top_k=50, top_p=0.95):
    """
    Generate text using the trained model
    
    Args:
        model: Trained MiniGPT model
        tokenizer: GPT-2 tokenizer
        prompt: Input text prompt
        max_length: Maximum length of generated text
        temperature: Sampling temperature (higher = more random)
        top_k: Top-k sampling
        top_p: Top-p (nucleus) sampling
    
    Returns:
        Generated text string
    """
    model.eval()
    
    # Tokenize input
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    
    with torch.no_grad():
        for _ in range(max_length):
            # Get model predictions
            outputs = model(input_ids)
            logits = outputs[:, -1, :]  # Get logits for the last token
            
            # Apply temperature
            logits = logits / temperature
            
            # Apply top-k filtering
            if top_k > 0:
                top_k_logits, top_k_indices = torch.topk(logits, top_k)
                logits_filtered = torch.full_like(logits, float('-inf'))
                logits_filtered.scatter_(1, top_k_indices, top_k_logits)
                logits = logits_filtered
            
            # Apply top-p filtering
            if top_p < 1.0:
                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
                
                # Remove tokens with cumulative probability above the threshold
                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
                sorted_indices_to_remove[..., 0] = 0
                
                indices_to_remove = sorted_indices_to_remove.scatter(1, sorted_indices, sorted_indices_to_remove)
                logits[indices_to_remove] = float('-inf')
            
            # Sample from the filtered distribution
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            
            # Add the new token to input_ids
            input_ids = torch.cat([input_ids, next_token], dim=-1)
            
            # Check if we've hit the EOS token
            if next_token.item() == tokenizer.eos_token_id:
                break
            
            # Prevent sequences from getting too long for memory
            if input_ids.shape[1] > model.max_seq_len:
                break
    
    # Decode the generated sequence
    generated_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    return generated_text

def chat():
    """
    Simple chat interface for interacting with the model
    """
    print("Mini-ChatGPT is ready! Type 'quit' to exit.")
    print("Note: This is a simple model trained on limited data, so responses may be basic.")
    print("-" * 60)
    
    while True:
        try:
            # Get user input
            user_input = input("\nYou: ").strip()
            
            if user_input.lower() in ['quit', 'exit', 'bye']:
                print("Goodbye!")
                break
            
            if not user_input:
                continue
            
            # Generate response
            print("AI: ", end="", flush=True)
            response = generate_text(
                model, 
                tokenizer, 
                user_input, 
                max_length=50,  # Keep responses short
                temperature=0.8,
                top_k=50,
                top_p=0.95
            )
            
            # Extract just the new part (after the input)
            if response.startswith(user_input):
                ai_response = response[len(user_input):].strip()
            else:
                ai_response = response.strip()
            
            print(ai_response if ai_response else "[Model generated empty response]")
            
        except KeyboardInterrupt:
            print("\nGoodbye!")
            break
        except Exception as e:
            print(f"Error: {e}")
            continue

# Test the generation function
print("Testing text generation...")
test_prompts = [
    "The weather today is",
    "Artificial intelligence is",
    "In the future, we will",
    "Programming is"
]

for prompt in test_prompts:
    generated = generate_text(model, tokenizer, prompt, max_length=30, temperature=0.8)
    print(f"Prompt: '{prompt}'")
    print(f"Generated: '{generated}'")
    print("-" * 40)

## 7. Interactive Chat Session

In [ ]:
# Start the chat interface
# Uncomment the line below to start chatting with your mini-GPT!
# chat()

# For demonstration purposes, let's show some example interactions
print("Example interactions with Mini-ChatGPT:")
print("=" * 50)

example_prompts = [
    "Hello, how are you?",
    "What is machine learning?",
    "Tell me a story about",
    "The benefits of Python programming are",
    "Explain what a neural network"
]

for i, prompt in enumerate(example_prompts, 1):
    print(f"\nExample {i}:")
    print(f"User: {prompt}")
    
    response = generate_text(
        model, 
        tokenizer, 
        prompt, 
        max_length=40,
        temperature=0.8,
        top_k=50,
        top_p=0.95
    )
    
    # Extract the generated part
    if response.startswith(prompt):
        ai_response = response[len(prompt):].strip()
    else:
        ai_response = response.strip()
    
    print(f"AI: {prompt}{ai_response}")
    print("-" * 30)

## 8. Model Evaluation and Analysis

In [ ]:
# Model evaluation
print("Model Analysis:")
print("=" * 40)

# Memory usage
if torch.cuda.is_available():
    memory_allocated = torch.cuda.memory_allocated(device) / 1024**2  # MB
    memory_reserved = torch.cuda.memory_reserved(device) / 1024**2   # MB
    print(f"GPU Memory Allocated: {memory_allocated:.1f} MB")
    print(f"GPU Memory Reserved: {memory_reserved:.1f} MB")

# Model size analysis
print(f"\nModel Architecture:")
print(f"- Vocabulary Size: {model_config['vocab_size']:,}")
print(f"- Hidden Dimensions: {model_config['d_model']}")
print(f"- Number of Layers: {model_config['num_layers']}")
print(f"- Number of Attention Heads: {model_config['num_heads']}")
print(f"- Feed-Forward Dimension: {model_config['d_ff']}")
print(f"- Maximum Sequence Length: {model_config['max_seq_len']}")

print(f"\nTraining Statistics:")
print(f"- Dataset Size: {len(train_dataset):,} samples")
print(f"- Batch Size: {batch_size}")
print(f"- Training Steps: {step:,}")
print(f"- Final Loss: {final_avg_loss:.4f}")
print(f"- Final Perplexity: {final_perplexity:.2f}")

# Save model checkpoint (optional)
print(f"\nSaving model checkpoint...")
checkpoint = {
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'model_config': model_config,
    'training_loss': final_avg_loss,
    'training_steps': step,
    'epochs': num_epochs
}

torch.save(checkpoint, '/content/mini_chatgpt_checkpoint.pt')
print("Model saved as 'mini_chatgpt_checkpoint.pt'")

print("\n" + "=" * 50)
print("Mini-ChatGPT implementation completed!")
print("\nTo chat with your model, uncomment and run the chat() function above.")
print("\nNote: This is a basic implementation for educational purposes.")
print("For production use, consider:")
print("- More training data and longer training")
print("- Larger model architecture")
print("- Advanced techniques like RLHF")
print("- Better evaluation metrics")
print("- Safety and bias considerations")

## 9. Usage Instructions and Next Steps

### How to Use This Notebook:

1. **Run all cells sequentially** - The notebook is designed to be executed from top to bottom
2. **Monitor training** - Watch the loss and perplexity metrics during training
3. **Test generation** - The notebook includes automatic testing of text generation
4. **Interactive chat** - Uncomment the `chat()` function call to start an interactive session

### Customization Options:

- **Model size**: Adjust `d_model`, `num_layers`, `num_heads` in the model configuration
- **Training data**: Modify `sample_size` to use more/less data
- **Sequence length**: Change `max_seq_length` for longer/shorter contexts
- **Training duration**: Adjust `num_epochs` and learning parameters
- **Generation parameters**: Modify `temperature`, `top_k`, `top_p` for different text styles

### Limitations and Considerations:

- This is a simplified implementation for educational purposes
- The model is small and trained on limited data
- Responses may not be coherent or factually accurate
- No safety filtering or bias mitigation is implemented
- GPU memory constraints limit model size in Colab

### Possible Improvements:

- Use larger models with more parameters
- Train on more diverse and larger datasets
- Implement advanced training techniques (gradient accumulation, mixed precision)
- Add instruction following capabilities
- Implement safety and content filtering
- Add evaluation metrics (BLEU, ROUGE, etc.)
- Create a web interface for easier interaction